In [1]:
# Imports
import pyarrow as pa
import pandas as pd

#D atei einlesen
df = pd.read_parquet("event_ano.parquet.gzip",engine="auto")
print("Data loaded.")
df.head(10)

Data loaded.


,Unnamed: 0,event_timestamp,message_number,message_category,message_severity,an_title,an_description,an_line,an_cell,an_robot
0,0,2016-07-04 16:19:35,10010,Operational,Information,None,None,line_0,cell_0,robot_0
1,1,2016-07-04 16:19:35,10017,Operational,Information,None,None,line_0,cell_0,robot_0
2,2,2016-07-04 16:19:35,10016,Operational,Information,None,None,line_0,cell_0,robot_0
3,3,2016-07-04 16:19:34,10015,Operational,Information,None,None,line_0,cell_0,robot_0
4,4,2016-07-04 16:19:34,10012,Operational,Information,None,None,line_0,cell_0,robot_0
5,5,2016-07-04 16:19:34,10011,Operational,Information,None,None,line_0,cell_0,robot_0
6,6,2016-07-04 16:19:33,10017,Operational,Information,None,None,line_0,cell_0,robot_0
7,7,2016-07-04 16:19:33,10016,Operational,Information,None,None,line_0,cell_0,robot_0
8,8,2016-07-04 16:19:32,40168,Program,Warning,None,None,line_0,cell_0,robot_0
9,9,2016-07-04 16:19:32,134010,Paint,Error,None,None,line_0,cell_0,robot_0


In [2]:
# Spalte Event Timestamp zu datetime64-Typ umwandeln
df['event_timestamp'] = pd.to_datetime(df["event_timestamp"])
df.dtypes

Unnamed: 0                   int64
event_timestamp     datetime64[ns]
message_number               int64
message_category            object
message_severity            object
an_title                    object
an_description              object
an_line                     object
an_cell                     object
an_robot                    object
dtype: object

In [3]:
#Datenframe generieren, der nach Zeit sortiert ist. (Beginnend mit ältestem Eintrag zuerst
df_sorted = df.sort_values(by="event_timestamp")
print("Data sorted.")
df_sorted.head(1)
#df_sorted.tail(5)
print(df_sorted.loc[0])
#print(df_sorted.iloc[0])

Data sorted.
Unnamed: 0                            0
event_timestamp     2016-07-04 16:19:35
message_number                    10010
message_category            Operational
message_severity            Information
an_title                           None
an_description                     None
an_line                          line_0
an_cell                          cell_0
an_robot                        robot_0
Name: 0, dtype: object


In [4]:
#Einen Fehlercode als gerade zu untersuchenden Fehler festlegen.
#Anm.: Hier muss später eine Liste aller Errortypen erstellt werden, die automatisch übergeben wird, 
#um zu verhindern, dass alle Fehler von Hand eingetippt werden müssen
current_observed_error = 20205

#Es wird eine Liste erstellt in welcher gespeichert wird an welcher Stelle [ID] im originalen und nach Zeit
#sortierten Datensatz der aktuelle Error auftritt
position_of_errors = df_sorted.index[df_sorted["message_number"] == current_observed_error].tolist()

#Als Beispiel die ersten 9 Stellen ausgeben.
print(position_of_errors[:9])

#Wie oft kommt der Fehler vor?
print(len(position_of_errors))
print("done.")

[293610, 293626, 293585, 293670, 293691, 293652, 293668, 293649, 293608]
124611
done.


In [5]:
#Durch einsetzen der Error-Positionsliste in das originale DataFrame werden mittels iloc
#alle Fehler dieses Typs angezeigt.
df_sorted.loc[position_of_errors[:]][0:15]
df_sorted.iloc[0]


Unnamed: 0                       293611
event_timestamp     2015-05-04 16:04:00
message_number                    10012
message_category            Operational
message_severity            Information
an_title                           None
an_description                     None
an_line                          line_6
an_cell                         cell_11
an_robot                       robot_21
Name: 293611, dtype: object

In [6]:
#Algorithmus zur Untersuchung:

before_error_messages = []
type(before_error_messages)
for x in position_of_errors[:3000]: #!!!!!HIER NUR DIE ersten 3000 Errors untersucht
    y=1
    for y in range(1,20): #Range => Soviele nachrichten zuvor werden gerpüft
        #print("x: "+str(x)+" y: "+str(y))
        #print("Hauptnachricht: "+str(df_sorted.loc[x].an_robot)+" x: "+str(x)+" "+str(df_sorted.loc[x].message_number))
        #print("Vorherige Nachricht: "+str(df_sorted.loc[x-y].an_robot)+" x-y: "+str(x-y)+" "+str(df_sorted.loc[x-y].message_number))
        #print("Datum HN:"+str(df_sorted.iloc[x].event_timestamp))
        #print("Datum vorh.:"+str(df_sorted.iloc[x-y].event_timestamp))
        #print(df_sorted.iloc[x].event_timestamp - df_sorted.iloc[x-y].event_timestamp)
        # Wenn der Roboter gleich ist, wie in der Nachricht zuvor UND die Message Severity KEIN Error ist, kann die Nummer
        # der dem Fehler vorausgehenden Nachricht gespeichert werden.
        #Zuvor soll allerdings zusätzlich geprüft werden, wie weit das Error-Ereignis und die Nachricht zuvor auseinander liegen.
        #Hier veranschlage ich einen Zeitraum von 60 Minuten. 
        if ((df_sorted.loc[x].an_robot == df_sorted.loc[x-y].an_robot) and 
            (df_sorted.loc[x-y].message_severity != "Error") and
             ((df_sorted.loc[x].event_timestamp - df_sorted.loc[x-y].event_timestamp) <= pd.Timedelta('1 hours'))):

                #print("Roboter sind gleich. Füge Nummer "+str(df_sorted.loc[x-y].message_number)+" der Liste hinzu.")
                before_error_messages.append(df_sorted.loc[x-y].message_number)
                break
        else:
            #print("Roboter nicht gleich, gehe eine Nachricht weiter zurück.")
            y=y+1
        
#print("-----------------------Check für "+str(x) +" vorbei---------------")                 




In [7]:
len(before_error_messages)

2801

In [16]:
df2 = pd.DataFrame(before_error_messages)
df3 = pd.Series(before_error_messages)
df3.value_counts()

10010    1834
10012     490
71439     187
71518      86
40538      56
10230      31
10011      16
71444      16
20314      15
41630      14
20481      12
10129       8
10015       8
10125       7
10053       5
71276       5
10156       2
50024       2
50438       1
10231       1
10052       1
10270       1
40708       1
10017       1
10271       1
dtype: int64